In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time

from bs4 import BeautifulSoup

import pandas as pd
import openpyxl

In [ ]:
@Bitcoin
@CoinMarketCap
@elonmusk
@CoinDesk
@CoinDeskMarkets
@CoinDeskData
@coinbase
@CoinbaseSupport

In [2]:
def prs_twtr(path):
    with open(path, 'r') as f:
        contents = f.read()
        page_soup = BeautifulSoup(contents, 'html.parser')

    # finds each product from the store page
    containers = page_soup.findAll("article")

    lista_tweet = []
    lista_data = []
    lista_reply = []
    lista_commenti = []
    lista_retweet = []
    lista_like = []

    for cnt in containers:
        # tweet
        try:
            tweet = cnt.findAll("div", {"class": "css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0"}) 
            
            try:
                ls = tweet[0].findAll("span")
            except LookupError:
                ls = "****"

            tmp1 = []
            
            for i in range(len(ls)):
                try:
                    tmp1.append(ls[i].text)
                except AttributeError:
                    tmp1.append(ls)

            string1 = " ".join(tmp1)

            lista_tweet.append(string1)

        except LookupError:
            lista_tweet.append("****")

        # data del tweet
        try:
            data = cnt.findAll("time")
            dict = data[0].attrs
            ls1 = str(dict.values())
            lista_data.append(ls1[14:-8])

        except LookupError:
            lista_data.append("****")

        # reply
        try:
            reply = cnt.findAll("div", {
                                "class": "css-901oao r-111h2gw r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-qvutc0"})
            ls2 = reply[0].findAll("a")
            tmp = []
            for x in range(len(ls2)):
                tmp.append(ls2[x]['href'])
                
            string = " ".join(tmp)
            lista_reply.append(string)

        except LookupError:
            lista_reply.append(" ")

        # num commenti
        try:
            commenti = cnt.findAll(
                "div", {"class": "css-1dbjc4n r-18u37iz r-1h0z5md"})
            ls3 = commenti[0].span.text
            lista_commenti.append(ls3)
            
        except LookupError:
            lista_commenti.append('****')

        # num retweet
        try:
            ls4 = commenti[1].span.text

            lista_retweet.append(ls4)
        except LookupError:
            lista_retweet.append('****')

        # num like
        try:
            ls5 = commenti[2].span.text

            lista_like.append(ls5)
        except LookupError:
            lista_like.append('****')

    cols = ["tweet", "data", "reply", "commenti", "retweet", "like"]
    df = pd.DataFrame(list(zip(lista_tweet, lista_data, lista_reply, lista_commenti, lista_retweet, lista_like)),
                      columns=cols)

    return(clean_parsed(df))

In [7]:
big_fishes = ['Bitcoin','CoinMarketCap','elonmusk','CoinDesk','CoinDeskMarkets','CoinDeskData','coinbase','CoinbaseSuppor']
fish = 'coinmarketcap'

In [5]:
def clean_parsed(df):
    df['data'] = pd.to_datetime(df['data'].str.replace('T',' '))
    df.set_index('data', drop=True, inplace=True)
    
    return df

----------

In [8]:
%%time

options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': { 'images': 2 }}
options.add_experimental_option('prefs', prefs)

cols = ["tweet","data","reply","commenti","retweet","like" ]
final = pd.DataFrame([],columns=cols)
chrome_path = "/Users/matteoprandi/Desktop/chromedriver"
base_path = "/Users/matteoprandi/Desktop/tweet_test/"
source_path = base_path + "page_source.html"

ls_account=["CoinMarketCap"]
dt_since="2020-01-01"
dt_until="2021-01-01"

for fish in big_fishes:
    url = "https://twitter.com/search?q=From%3A" + fish + "%20since%3A"+ dt_since +"%20until%3A"+ dt_until + "&src=typed_query&f=live"

    driver = webdriver.Chrome(options=options, executable_path=chrome_path)
    driver.get(url)
    time.sleep(5)
    
    dummy = 0

    while True:
        with open(source_path, "w") as f:
            f.write(driver.page_source)

        df = prs_twtr(source_path)
        final = final.append(df)

        last_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, {})".format(last_height + 500))
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
                
        if (final.shape[0] - dummy) >= 500:
            driver.close()
            dt_until = str(final.index[-10].date())
            url = "https://twitter.com/search?q=From%3A" + fish + "%20since%3A"+ dt_since +"%20until%3A"+ dt_until + "&src=typed_query&f=live"
            driver = webdriver.Chrome(options=options, executable_path=chrome_path)
            driver.get(url)
            time.sleep(5)
            
            dummy = final.shape[0]

        if last_height == new_height:
            time.sleep(5)
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            if last_height == new_height:
                with open(source_path, "w") as f:
                    f.write(driver.page_source)
                    
                df = prs_twtr(source_path)
                final = final.append(df)
                final = final.drop_duplicates(subset='tweet', keep="first")
                final.to_csv(base_path + str(fish) + '.csv')
                driver.close()
                break

IndentationError: unexpected indent (<unknown>, line 16)

In [ ]:
%%time

options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': { 'images': 2 }}
options.add_experimental_option('prefs', prefs)

cols = ["tweet","data","reply","commenti","retweet","like" ]
final = pd.DataFrame([],columns=cols)
chrome_path = "/Users/matteoprandi/Desktop/chromedriver"
base_path = "/Users/matteoprandi/Desktop/tweet_test/"
source_path = base_path + "page_source.html"

ls_account=["CoinMarketCap"]
dt_since="2020-01-01"
dt_until="2021-01-01"

fish = 'coinmarketcap'

url = "https://twitter.com/search?q=From%3A" + fish + "%20since%3A"+ dt_since +"%20until%3A"+ dt_until + "&src=typed_query&f=live"

driver = webdriver.Chrome(options=options, executable_path=chrome_path)
driver.get(url)
time.sleep(5)

dummy = 0

while True:
    with open(source_path, "w") as f:
        f.write(driver.page_source)

    df = prs_twtr(source_path)
    final = final.append(df)

    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, {})".format(last_height + 500))
    time.sleep(1)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if (final.shape[0] - dummy) >= 500:
        driver.close()
        dt_until = str(final.index[-10].date())
        url = "https://twitter.com/search?q=From%3A" + fish + "%20since%3A"+ dt_since +"%20until%3A"+ dt_until + "&src=typed_query&f=live"
        driver = webdriver.Chrome(options=options, executable_path=chrome_path)
        driver.get(url)
        time.sleep(5)

        dummy = final.shape[0]

    if last_height == new_height:
        time.sleep(5)
        last_height = driver.execute_script("return document.body.scrollHeight")

        if last_height == new_height:
            with open(source_path, "w") as f:
                f.write(driver.page_source)

            df = prs_twtr(source_path)
            final = final.append(df)
            final = final.drop_duplicates(subset='tweet', keep="first")
            final.to_csv(base_path + str(fish) + '.csv')
            driver.close()
            break

In [10]:
t = pd.read_csv(base_path + 'CoinMarketCap.csv')
t.shape

(3478, 7)